In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import ast
import random
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from pprint import pprint
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.utils import resample

In [10]:
def plot_tags_count(data: pd.DataFrame | dict, zoom_max=10000):
    if isinstance(data, pd.DataFrame):
        data = Counter([item for sublist in data['labels'] for item in sublist])
    sorted_values_cntr = {k: v for k, v in sorted(data.items(), key=lambda item: item[1], reverse=True)}

    if zoom_max:
        sorted_values_cntr = {k: v for k, v in sorted_values_cntr.items() if v < zoom_max}

    plt.figure(figsize=(10, 10))
    plt.pie(
        sorted_values_cntr.values(),
        labels=sorted_values_cntr.keys(),
        autopct='%1.1f%%', startangle=0
    )
    plt.axis('equal')
    plt.title('Label Distribution in Test Set')

    plt.show()


def plot_variance_per_key(data: dict):
    plt.figure(figsize=(10, 10))
    plt.bar(data.keys(), data.values())
    plt.title('Label Distribution in Test Set')
    plt.show()

In [17]:
SRC_DIR = Path('/Volumes/external_drive')

try:
    _df = pd.read_csv('datasets/images_high_res_dataset.csv')
except FileNotFoundError:
    _df = pd.read_csv(SRC_DIR / 'images_high_res_dataset.csv')

print("Dataset loaded.")
_df['categories'] = _df['categories'].apply(ast.literal_eval)
_df['categories_suggestions'] = _df['categories_suggestions'].apply(ast.literal_eval)
print("Parsed list columns categories")

Dataset loaded.
Parsed list columns categories


In [18]:
df = _df.copy(True)
df.shape

(1344601, 8)

# Merge suggestions and categories

In [19]:
def merge_categories(row):
    categories = set(row['categories'])
    categories_suggestions = set(row['categories_suggestions'])
    categories_superset = {category.strip().lower() for category in categories.union(categories_suggestions)}
    return list(categories_superset)


df['labels'] = df['categories'].apply(lambda x: [category.strip().lower() for category in x])
df['labels'] = df.apply(merge_categories, axis=1)
df.drop(['categories_suggestions', 'categories'], axis=1, inplace=True)
df.shape

(1344601, 7)

# Purge nationalities from tags

In [20]:
nationality_tags_to_purge = {
    'african',
    'american',
    'arab',
    'argentina',
    'australian',
    'brazilian',
    'british',
    'canadian',
    'chinese',
    'colombian',
    'cuban',
    'czech',
    'dutch',
    'european',
    'filipina',
    'french',
    'german',
    'hungarian',
    'indian',
    'italian',
    'japanese',
    'korean',
    'mexican',
    'pinay',
    'polish',
    'russian',
    'spanish',
    'thai',
    'ukrainian',
    'venezuela',
    'white'
}
nationality_tags_to_purge = {n.lower() for n in nationality_tags_to_purge}
df['labels'] = df.labels.apply(lambda x: list(set(x) - nationality_tags_to_purge))
df.shape

(1344601, 7)

# Apply gallery mapping

In [21]:
from galleries_mapping import *

df_not_mapped = df.copy(True)


def gallery_mapping(row):
    labels = row.labels

    out = []
    for L in labels:
        _fetched = GALLERIES_MAP.get(L, None)
        if _fetched is remove_tag:
            continue
        elif _fetched is remove_gallery:
            return None
        elif isinstance(_fetched, list):
            out.extend(_fetched)
        elif _fetched is keep_tag:
            out.append(L.lower())

    return list(set(out))


df['labels'] = df.apply(gallery_mapping, axis=1)
df = df[df['labels'].notnull()]
df.shape

(1106872, 7)

# Dataframe clean up

In [22]:
def add_file_path_column(dataframe: pd.DataFrame):
    dataframe['file_path'] = dataframe['gallery_category'] + '/' + dataframe['gallery_name'] + '/' + dataframe[
        'filename']
    dataframe = dataframe.drop(
        ['gallery_category', 'gallery_name', 'filename'], axis=1
    )
    dataframe.reset_index(inplace=True, drop=True)
    return dataframe


df.drop(
    ['height', 'width'], axis=1, inplace=True
)

df = add_file_path_column(df)

# Balancing the dataset

In [23]:
label_counts = Counter(item for sublist in df['labels'] for item in sublist)
ascending_labels = [
    k for k, v in sorted(label_counts.items(), key=lambda item: item[1])
]
label_proportions = {
    k: v / len(df) for k, v in sorted(label_counts.items(), key=lambda item: item[1])
}

In [24]:
def get_balanced_df(og_df, max_samples=1000):
    def get_rows_with_label(dataframe, label):
        return dataframe[dataframe['labels'].apply(lambda x: label in x)]

    balanced_dfs = {label: pd.DataFrame() for label in label_counts.keys()}

    for label in tqdm(ascending_labels, total=len(ascending_labels), desc='Balancing dataset'):
        label_df = get_rows_with_label(og_df, label)

        label_df = resample(
            label_df,
            n_samples=max_samples,
            random_state=42
        )

        balanced_dfs[label] = label_df

    balanced_df = pd.concat(balanced_dfs).drop_duplicates(subset='file_path').reset_index(drop=True)

    return balanced_df

In [25]:
small_balanced_df = get_balanced_df(df, max_samples=1000)
medium_balanced_df = get_balanced_df(df, max_samples=3000)
final_balanced_df = df.copy()

Balancing dataset: 100%|██████████| 110/110 [00:25<00:00,  4.25it/s]


In [26]:
small_balanced_df.to_csv('datasets/small_file_paths.csv')
medium_balanced_df.to_csv('datasets/medium_file_paths.csv')
final_balanced_df.to_csv('datasets/all_file_paths.csv')

# One hot encoding

In [27]:
def dataframe_one_hot_encoding(dataframe: pd.DataFrame):
    all_labels = set(label for labels in dataframe['labels'] for label in labels)
    one_hot_encoded = pd.DataFrame()

    for label in tqdm(all_labels, total=len(all_labels), desc='One hot encoding'):
        one_hot_encoded[label] = dataframe['labels'].apply(lambda x: 1 if label in x else 0)

    out_df = pd.concat([dataframe, one_hot_encoded], axis=1)
    out_df.reset_index(drop=True, inplace=True)

    cols_sorted = sorted(list(out_df.columns))
    cols_sorted.remove("file_path")
    cols_sorted.remove("labels")

    return out_df[["file_path", "labels", *cols_sorted]]

In [28]:
small_final_df = dataframe_one_hot_encoding(small_balanced_df)
medium_final_df = dataframe_one_hot_encoding(medium_balanced_df)
final_df = dataframe_one_hot_encoding(df)

One hot encoding:  91%|█████████ | 100/110 [00:06<00:00, 14.58it/s]/var/folders/cq/qnm4dkj53rq748w0d5l1yz5h0000gn/T/ipykernel_55395/4168988818.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  one_hot_encoded[label] = dataframe['labels'].apply(lambda x: 1 if label in x else 0)
/var/folders/cq/qnm4dkj53rq748w0d5l1yz5h0000gn/T/ipykernel_55395/4168988818.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  one_hot_encoded[label] = dataframe['labels'].apply(lambda x: 1 if label in x else 0)
One hot encoding:  93%|█████████▎| 102/11

In [29]:
small_final_df.to_csv("datasets/small_one_hot.csv")
medium_final_df.to_csv("datasets/medium_one_hot.csv")
final_df.to_csv("datasets/all_one_hot.csv")

# AI GEN

In [30]:
from galleries_mapping import *

In [31]:
_mandatory_columns = [
    'age',
]
mandatory_columns = list(
    x.strip() for x in AI_GEN_TAGS[_mandatory_columns].values.flatten() if isinstance(x, str)
)
_negative_columns = [
    'negative', 'more_people'
]
negative_columns = list(
    x.strip() for x in AI_GEN_TAGS[_negative_columns].values.flatten() if isinstance(x, str)
)
tags_to_drop = {
    'blowjob', 'reality', 'hardcore', 'cumshot', 'cowgirl', 'ass fucking', 'doggy style', 'teen', 'lesbian', 'model',
    'big cock', 'anal', 'bdsm', 'bondage', 'fetish', 'face', 'kissing', 'handjob', 'facial', 'fingering', 'groupsex',
    'cum in mouth', 'old young', 'titjob', 'interracial', 'pussy licking', 'threesome', 'pov', 'femdom', 'christmas',
    'girlfriend', 'cosplay', 'facesitting', 'massage', 'deepthroat', 'strapon', 'cheating', 'humping', 'cum in pussy',
    'ass licking', 'creampie', 'ball licking', 'spanking', 'orgasm', 'double penetration', 'couple', 'family',
    'anal gape', 'bbc', 'party', 'schoolgirl', 'fisting', 'missionary', 'squirting', 'pissing', 'gangbang', 'old man',
    'ffm', 'cuckold', 'seduction', 'tribbing', 'orgy', 'flexible', 'cfnm', 'footjob', 'blowbang', 'pegging', 'pregnant',
    'swingers', 'gloryhole', 'caught', 'college', 'yoga', 'casting', 'stripper', 'step sister', 'voyeur', 'mmf',
    'bukkake', 'gyno', 'small cock', 'babysitter', 'cheerleader', 'cum swapping', 'bisexual', 'goth', 'braces', 'pawg',
    'pretty', 'pigtails', 'emo', 'latex', 'babe', 'step brother', 'twink', 'shemale', 'ballerina', 'twins', 'pornstar',
    'model', 'latex', 'emo', 'latex', 'babe', 'leather', 'pigtails', 'halloween', 'wedding',
    'tall', 'doctor', 'vintage', 'rough sex', 'sex', 'gym', 'sandals', 'big woman',
}

In [32]:
def filter_positives(row):
    labels = row.labels
    labels = [L.lower() for L in labels]
    for L in labels:
        if L in mandatory_columns:
            return labels


def filter_negatives(row):
    labels = row.labels
    labels = [L.lower() for L in labels]

    for L in labels:
        if L in negative_columns:
            return None
        if L in tags_to_drop:
            return None
    return labels


ai_df = df_not_mapped.copy()
ai_df['labels'] = ai_df.apply(filter_positives, axis=1)
ai_df = ai_df[ai_df['labels'].notnull()]
print(ai_df.shape)

ai_df['labels'] = ai_df.apply(filter_negatives, axis=1)
ai_df = ai_df[ai_df['labels'].notnull()]
ai_df.reset_index(inplace=True, drop=True)
print(ai_df.shape)

(411065, 7)
(37734, 7)


In [33]:
def ai_gen_gallery_mapping(row):
    labels = row.labels

    out = []
    for L in labels:
        _fetched = AI_GEN_GALLERIES_MAP.get(L, None)
        if _fetched is remove_tag:
            continue
        elif _fetched is remove_gallery:
            return None
        elif isinstance(_fetched, list):
            out.extend(_fetched)
        elif _fetched is keep_tag:
            out.append(L.lower())

    return list(set(out))


ai_df['labels'] = ai_df.apply(ai_gen_gallery_mapping, axis=1)
ai_df = ai_df[ai_df['labels'].notnull()]
ai_df.drop(
    ['height', 'width'], axis=1, inplace=True
)
ai_df.shape

(37734, 5)

In [34]:
ai_df = add_file_path_column(ai_df)
ai_df.shape

(37734, 3)

In [36]:
ai_df.to_csv("datasets/ai_gen.csv")

# Dataset inspection

In [37]:
ai_gen = pd.read_csv('datasets/ai_gen.csv', index_col=0)
small_one_hot = pd.read_csv('datasets/small_one_hot.csv', index_col=0)
medium_one_hot = pd.read_csv('datasets/medium_one_hot.csv', index_col=0)

In [49]:
missing_paths = []
missing = 0

for file_path in medium_one_hot['file_path'].to_list():
    file_path = Path(file_path)
    json_path = SRC_DIR / file_path.parent / 'cropped' / (file_path.stem + '.json')
    if not json_path.exists():
        missing_paths.append((SRC_DIR / file_path).as_posix())
        missing += 1

In [50]:
missing_df = pd.DataFrame(missing_paths, columns=['file_path'])

42108

In [54]:
missing_df.to_csv('datasets/missing_paths.csv')